In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

import folium
import folium.plugins as plugins

import geohash2

In [ ]:
hdfs_port = "hdfs://orion11:26990"
# data_path = "/nam_s/nam_201501_s*"
data_path = "/nam_s/*"
# data_path = "/sample/nam_tiny*"

In [ ]:
feats = []
f = open('../features.txt')
for line_num, line in enumerate(f):
    if line_num == 0:
        # Timestamp
        feats.append(StructField(line.strip(), LongType(), True))
    elif line_num == 1:
        # Geohash
        feats.append(StructField(line.strip(), StringType(), True))
    else:
        # Other features
        feats.append(StructField(line.strip(), FloatType(), True))
    
schema = StructType(feats)

In [4]:
%%time

df = spark.read.format('csv').option('sep', '\t').schema(schema).load(f'{hdfs_port}{data_path}')

CPU times: user 1.71 ms, sys: 1.13 ms, total: 2.84 ms
Wall time: 5.76 s


## Selecting Geohashes

We do a little programming to create a set of geohashes. The suffix values are a "list" of suffixes to attach to it's parent. For instance, each suffix in suffix_9qb will be appended to "9qb".  

![geohashes](img/bay_area_geohashes.PNG)

In [5]:
suffix_9qb = "6df9c8b1"
suffix_9qc = "10"
suffix_9q8 = "xzyvug"
suffix_9q9 = "pnjhmk57"

bay_area_hashes = {*["9qb" + letter for letter in suffix_9qb],
                  *["9qc" + letter for letter in suffix_9qc],
                  *["9q8" + letter for letter in suffix_9q8],
                  *["9q9" + letter for letter in suffix_9q9]}

print(bay_area_hashes)

{'9q8y', '9q8u', '9qbc', '9q9k', '9q8g', '9q9m', '9q9p', '9q8v', '9q97', '9q9h', '9q9j', '9q8x', '9qb9', '9q8z', '9qbd', '9q9n', '9q95', '9qb6', '9qb1', '9qc1', '9qbb', '9qc0', '9qbf', '9qb8'}


## Filtering

Now we do some filtering to the bay_area hashes.

In [6]:
%%time
dg = df.select("Geohash", "land_cover_land1_sea0_surface", "visibility_surface", df.Geohash.substr(1, 4).alias("front_hash"), "geopotential_height_cloud_base", "geopotential_height_surface")

dg = dg[dg.front_hash.isin(*bay_area_hashes)]

CPU times: user 12 ms, sys: 2.47 ms, total: 14.5 ms
Wall time: 623 ms


## SQL Command

Now I check for if the clouds are lower then the land height and if that results in a decrese in visibility. I also make sure I'm looking for land locations.

I count the number of foggy days vs the total number of data points for that location and that value, "foggy_ratio", is the percentage of times that it's foggy in that area.

In [7]:
dg.createOrReplaceTempView("nam_small")

sql_query =f'''
SELECT 
    *, 
    foggy_days/counts AS foggy_ratio 
    FROM(
        SELECT substr(Geohash, 1, 5) AS geoloc,
            AVG(visibility_surface) AS vis_surf_avg,
            SUM(CASE WHEN geopotential_height_cloud_base <= geopotential_height_surface AND visibility_surface < 24221 THEN 1 ELSE 0 END) AS foggy_days,
            SUM(1) as counts
        FROM nam_small 
        WHERE land_cover_land1_sea0_surface=1
        GROUP BY substr(Geohash, 1, 5)) AS t2
ORDER BY foggy_ratio ASC, vis_surf_avg DESC, foggy_days ASC, counts ASC
'''

visibilities = spark.sql(sql_query)

print(sql_query)


SELECT 
    *, 
    foggy_days/counts AS foggy_ratio 
    FROM(
        SELECT substr(Geohash, 1, 5) AS geoloc,
            AVG(visibility_surface) AS vis_surf_avg,
            SUM(CASE WHEN geopotential_height_cloud_base <= geopotential_height_surface AND visibility_surface < 24221 THEN 1 ELSE 0 END) AS foggy_days,
            SUM(1) as counts
        FROM nam_small 
        WHERE land_cover_land1_sea0_surface=1
        GROUP BY substr(Geohash, 1, 5)) AS t2
ORDER BY foggy_ratio ASC, vis_surf_avg DESC, foggy_days ASC, counts ASC



## Analysis

Now that I have the data binned up, I can convert it to a pandas data frame and convert the geohash values to longitude and latitudes and plot those.

In [8]:
%%time
p_df = visibilities.toPandas()

CPU times: user 26.1 ms, sys: 8.45 ms, total: 34.5 ms
Wall time: 3min 7s


In [9]:
def to_lat(geohash):
    return geohash2.decode(geohash)[0]

def to_lon(geohash):
    return geohash2.decode(geohash)[1]

In [10]:
p_df['lat'] = p_df.apply(lambda row: to_lat(row.geoloc), axis=1)
p_df['lon'] = p_df.apply(lambda row: to_lon(row.geoloc), axis=1)
p_df = p_df.sort_values(['foggy_ratio', 'counts'], ascending=[1, 0])
h_10 = p_df.head(10)
h_10

,geoloc,vis_surf_avg,foggy_days,counts,foggy_ratio,lat,lon
0,9qc0v,23701.345179,25,404,0.061881,38.1,-122.1
1,9q97d,23233.363643,30,422,0.071090,37.2,-121.9
2,9q9hp,23440.124063,31,423,0.073286,37.3,-122.
3,9qc0q,24002.255109,32,430,0.074419,38.,-122.1
4,9q95r,23527.074557,33,438,0.075342,37.2,-122.
5,9q9nk,23079.967669,33,409,0.080685,37.7,-122.1
6,9q9ku,21960.097769,32,394,0.081218,37.4,-121.8
7,9q9jv,23745.016297,35,425,0.082353,37.6,-122.1
8,9q9nx,22922.526431,34,412,0.082524,37.7,-122.
9,9q9jn,23637.438335,35,420,0.083333,37.5,-122.1


## Results

We can see that all of the top 10 locations are far inland and away from where the cool ocean air meets the warm land air.

![result](img/res_escape_fog.PNG)

### San Francisco Coords 37.7749° N, 122.4194° W

In [ ]:
sf_coords = (37.7749, -122.4194)

In [ ]:
m = folium.Map(location=sf_coords, zoom_start=8)

def add_marker(folium_map, row):
    coords = geohash2.decode(row.geoloc)
    folium.Marker([float(coords[0]), float(coords[1])], popup=f'Foggy Ratio: {row.foggy_ratio}').add_to(folium_map)
    print(coords)

h_10.apply(lambda row: add_marker(m, row), axis=1)

m